# Space X Falcon 9 First Stage Landing Prediction
### Data wrangling
![Falcon 9](../images/falcon9.webp)
### Objective:
* Perform Exploratory Data Analysis (EDA)
* Determine Training Labels.
### Scenario:
* In the data set, there are several different cases where the booster did not land successfully. 
* Sometimes a landing was attempted but failed due to an accident; for example, 
     * True Ocean means the mission outcome was successfully landed to a specific region of the ocean while False Ocean means the mission outcome was unsuccessfully landed to a specific region of the ocean. 
     * True RTLS means the mission outcome was successfully landed to a ground pad False RTLS means the mission outcome was unsuccessfully landed to a ground pad.
     * True ASDS means the mission outcome was successfully landed on a drone ship False ASDS means the mission outcome was unsuccessfully landed on a drone ship.
* In this lab I will mainly convert those outcomes into Training Labels with **1** means the booster successfully landed **0** means it was unsuccessful.

![Space X](https://camo.githubusercontent.com/a0184ea6ee7174857c755b964345c82ec9556f17e74fd5f6a49b5937711fd60f/68747470733a2f2f63662d636f75727365732d646174612e73332e75732e636c6f75642d6f626a6563742d73746f726167652e617070646f6d61696e2e636c6f75642f49424d446576656c6f706572536b696c6c734e6574776f726b2d445330373031454e2d536b696c6c734e6574776f726b2f6170692f496d616765732f6c616e64696e675f312e676966 "Falcon 9 first stage will land successfully")



# Data Analysis

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../csv/dataset_part_1.csv')
df.head(10)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1.0,2010-06-04,Falcon 9,6123.547647,LEO,CCSFS SLC 40,None None,1.0,0.0,0.0,0.0,NaN,1.0,0.0,B0003,-80.577366,28.561857
1,2.0,2012-05-22,Falcon 9,525.000000,LEO,CCSFS SLC 40,None None,1.0,0.0,0.0,0.0,NaN,1.0,0.0,B0005,-80.577366,28.561857
2,3.0,2013-03-01,Falcon 9,677.000000,ISS,CCSFS SLC 40,None None,1.0,0.0,0.0,0.0,NaN,1.0,0.0,B0007,-80.577366,28.561857
3,4.0,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1.0,0.0,0.0,0.0,NaN,1.0,0.0,B1003,-120.610829,34.632093
4,5.0,2013-12-03,Falcon 9,3170.000000,GTO,CCSFS SLC 40,None None,1.0,0.0,0.0,0.0,NaN,1.0,0.0,B1004,-80.577366,28.561857
5,6.0,2014-01-06,Falcon 9,3325.000000,GTO,CCSFS SLC 40,None None,1.0,0.0,0.0,0.0,NaN,1.0,0.0,B1005,-80.577366,28.561857
6,7.0,2014-04-18,Falcon 9,2296.000000,ISS,CCSFS SLC 40,True Ocean,1.0,0.0,0.0,1.0,NaN,1.0,0.0,B1006,-80.577366,28.561857
7,8.0,2014-07-14,Falcon 9,1316.000000,LEO,CCSFS SLC 40,True Ocean,1.0,0.0,0.0,1.0,NaN,1.0,0.0,B1007,-80.577366,28.561857
8,9.0,2014-08-05,Falcon 9,4535.000000,GTO,CCSFS SLC 40,None None,1.0,0.0,0.0,0.0,NaN,1.0,0.0,B1008,-80.577366,28.561857
9,10.0,2014-09-07,Falcon 9,4428.000000,GTO,CCSFS SLC 40,None None,1.0,0.0,0.0,0.0,NaN,1.0,0.0,B1011,-80.577366,28.561857


In [3]:
# Identifying and calculating the percentage of the missing values in each attribute
df.isnull().sum()/df.count()*100

FlightNumber       1.111111
Date               1.111111
BoosterVersion     1.111111
PayloadMass        0.000000
Orbit              1.111111
LaunchSite         1.111111
Outcome            1.111111
Flights            1.111111
GridFins           1.111111
Reused             1.111111
Legs               1.111111
LandingPad        42.187500
Block              1.111111
ReusedCount        1.111111
Serial             1.111111
Longitude          1.111111
Latitude           1.111111
dtype: float64

In [4]:
# Identifying numerical and categorical columns:
df.dtypes

FlightNumber      float64
Date               object
BoosterVersion     object
PayloadMass       float64
Orbit              object
LaunchSite         object
Outcome            object
Flights           float64
GridFins          float64
Reused            float64
Legs              float64
LandingPad         object
Block             float64
ReusedCount       float64
Serial             object
Longitude         float64
Latitude          float64
dtype: object

### Calculating the number of launches on each site
#### Space X launch facilities: 
 * [Cape Canaveral Space](https://en.wikipedia.org/wiki/List_of_Cape_Canaveral_and_Merritt_Island_launch_sites?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01) Complex 40 **VAFB SLC 4E**,
 * Vandenberg Air Force Base Space Launch Complex 4E (**SLC-4E**), 
 * Kennedy Space Center Launch Complex 39A **KSC LC 39A**.

In [5]:
# determining the number of launches on each site:
df['LaunchSite'].value_counts()    # The column LaunchSite places the location of each Launch

LaunchSite
CCSFS SLC 40    55
KSC LC 39A      22
VAFB SLC 4E     13
Name: count, dtype: int64